## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Weather,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Sorong,ID,2022-03-02 19:49:04,-0.8833,131.2500,Light Rain,80.29,79,100,2.80
1,1,Batemans Bay,AU,2022-03-02 19:49:05,-35.7167,150.1833,Light Rain,68.20,92,98,10.89
2,2,Roald,NO,2022-03-02 19:49:05,62.5833,6.1333,Broken Clouds,39.33,100,75,3.44
3,3,San Cristobal,VE,2022-03-02 19:49:06,7.7669,-72.2250,Broken Clouds,78.98,39,65,5.46
4,4,Pochutla,MX,2022-03-02 19:49:07,15.7432,-96.4661,Clear Sky,88.83,43,3,10.29


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask customer for input on min/max temperature values
min_temp = float(input("What would you like to set as the 'Minimum' temperature for your trip?"))
max_temp = float(input("What would you like to set as the 'Maximum' temperature for your trip?"))

What would you like to set as the 'Minimum' temperature for your trip?55
What would you like to set as the 'Maximum' temperature for your trip?75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       172
City          172
Country       167
Date          172
Lat           172
Lng           172
Weather       172
Max Temp      172
Humidity      172
Cloudiness    172
Wind Speed    172
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID       167
City          167
Country       167
Date          167
Lat           167
Lng           167
Weather       167
Max Temp      167
Humidity      167
Cloudiness    167
Wind Speed    167
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Weather", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Weather,Max Temp,Lat,Lng,Hotel Name
1,Batemans Bay,AU,Light Rain,68.20,-35.7167,150.1833,
6,Cape Town,ZA,Few Clouds,70.12,-33.9258,18.4232,
11,San Quintin,MX,Overcast Clouds,70.52,30.4833,-115.9500,
12,Hermanus,ZA,Clear Sky,69.91,-34.4187,19.2345,
26,Fortuna,US,Broken Clouds,65.93,40.5982,-124.1573,
28,Jinka,ET,Light Rain,72.75,5.6500,36.6500,
31,Airai,TL,Few Clouds,60.42,-8.9266,125.4092,
41,Carnarvon,AU,Clear Sky,68.07,-24.8667,113.6333,
45,Saldanha,ZA,Clear Sky,69.85,-33.0117,17.9442,
46,East London,ZA,Overcast Clouds,73.60,-33.0153,27.9116,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Places API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print(f"Hotel not found in {row['City']}. Skipping {row['City']}.")       

Hotel not found in Jinka. Skipping Jinka.
Hotel not found in Airai. Skipping Airai.
Hotel not found in Adrar. Skipping Adrar.
Hotel not found in Marzuq. Skipping Marzuq.
Hotel not found in Taoudenni. Skipping Taoudenni.
Hotel not found in Baisha. Skipping Baisha.
Hotel not found in Lodja. Skipping Lodja.
Hotel not found in Masuguru. Skipping Masuguru.
Hotel not found in Abu Samrah. Skipping Abu Samrah.
Hotel not found in Lahij. Skipping Lahij.
Hotel not found in Hidalgotitlan. Skipping Hidalgotitlan.
Hotel not found in Beloha. Skipping Beloha.
Hotel not found in Rudbar. Skipping Rudbar.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
clean_hotel_df.count()

City          154
Country       154
Weather       154
Max Temp      154
Lat           154
Lng           154
Hotel Name    154
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30,0), zoom_level=1.3)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))